In [1]:
%cd ../

/u02/thanhhm/research-idea


In [2]:
import time

from plasma.parallel_processing import communicators, queues
from plasma.parallel_processing.communicators.tree_flow import TreeFlow
from plasma.functional import partials, AutoPipe

/u02/thanhhm/research-idea/.env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute(x):
    time.sleep(2)
    return ['a', 'b']

def compute2(x):
    time.sleep(2)
    return x + 'c'

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, tmp):
        return tmp + 'd'

In [4]:
out_queue= queues.TransferQueue()

class Flow(TreeFlow):

    def __init__(self):
        super().__init__()

        self.compute = communicators.processor.Yield(compute)
        self.compute2 = compute2
        self.compute3 = ComputeClass(1, 2)
        self.printer1 = partials(print, 'step 1')
        self.printer2 = partials(print, 'step 2')

        self.register_chains(
            (queues.ProcessQueue(5), None, 'compute'),
            (queues.ProcessQueue(5), 'compute', 'compute2'),
            (queues.TransferQueue(), 'compute', 'printer1'),
            (queues.TransferQueue(), 'compute2', 'printer2'),
            (queues.ProcessQueue(), 'compute', 'compute3'),
            (out_queue, 'compute3', None),
        )

flow = Flow()
flow

[ProcessQueue(runner=1)]

  |-(compute:<function compute at 0x7fabd3c458a0>)

    |-[ProcessQueue(runner=1)]

      |-(compute2:<function compute2 at 0x7fabd3a97a60>)

        |-[TransferQueue(runner=1)]

          |-(printer2:print(step 2,*,**))

    |-[TransferQueue(runner=1)]

      |-(printer1:print(step 1,*,**))

    |-[ProcessQueue(runner=1)]

      |-(compute3:ComputeClass)

        |-[TransferQueue(runner=1)]

In [5]:
out_queue.register_callback(print).run()

TransferQueue()

ad
ad
ad
bd
ad
bd
bd
ad
bd
bd
ad
bd
ad
bd
ad
ad
ad
bd
bd
bd


In [6]:
with flow:
   for q in flow.inputs.values():
      for i in range(10):
         q.put(i)

step 1 a
step 1 a
step 1 a
step 1 a
step 1 b
step 1 b
step 1 b
step 1 b
step 1 a
step 1 b
step 2 ac
step 1 a
step 1 b
step 1 a
step 1 b
step 1 a
step 1 b
step 1 a
step 1 b
step 1 a
step 1 b
step 2 ac
step 2 ac
step 2 ac
step 2 bc
step 2 bc
step 2 bc
step 2 bc
step 2 ac
step 2 bc
step 2 ac
step 2 bc
step 2 ac
step 2 bc
step 2 ac
step 2 bc
step 2 ac
step 2 bc
step 2 ac
step 2 bc


In [7]:
flow.inputs

{'compute': ProcessQueue(
   persistent=False,
   n=5,
 )}

In [8]:
flow.outputs

{'compute3': TransferQueue()}

In [9]:
out_queue.running

True